In [1]:
import numpy as np

nodes = []
for i in range(15000):
    years_of_experience = np.random.randint(0, 16) / 15.0
    degree_level = np.random.randint(1, 5)
    university_power = 0.0
    degree_relevancy = 0.0
    if degree_level > 1:
        university_power = np.random.randint(1, 11) / 10.0
        degree_relevancy = np.random.randint(1, 11) / 10.0
    degree_level /= 4.0

    performer_score_loc = (years_of_experience + degree_level + degree_relevancy + university_power) / 4.0
    performer_score = max(0.0, min(1.0, np.random.normal(performer_score_loc, 0.25)))
    protected_group = np.random.randint(0, 2)
    nodes.append(np.array([years_of_experience, degree_level, degree_relevancy, university_power, performer_score, protected_group]))
nodes = np.array(nodes)

In [2]:
from numpy.random import choice
edges = {}
performer_scores = nodes[:, 4]
for i in range(nodes.shape[0]):
    cur_num_edges = np.random.randint(5, 21)
    cur_protected_group = nodes[i][5]
    weights = np.where(nodes[:,5] == cur_protected_group, 2, 1) * performer_scores
    probs = weights / weights.sum()
    draw = choice(list(range(nodes.shape[0])), cur_num_edges,
              p=probs)
    for j in range(draw.shape[0]):
        suggested_edge_node = draw[j]
        if suggested_edge_node < i:
            if suggested_edge_node not in edges:
                edges[suggested_edge_node] = []
            edges[suggested_edge_node].append(i)
        else:
            if i not in edges:
                edges[i] = []
            edges[i].append(suggested_edge_node)
for key, value in edges.items():
    edges[key] = list(set(value))

In [3]:
print(nodes)

[[0.26666667 0.75       0.4        0.5        0.37617686 0.        ]
 [0.86666667 0.25       0.         0.         0.08900358 1.        ]
 [0.53333333 0.5        1.         0.3        0.52672275 0.        ]
 ...
 [0.13333333 1.         0.9        0.9        1.         0.        ]
 [0.33333333 0.75       0.5        0.3        0.34970352 0.        ]
 [0.2        0.75       0.8        0.5        0.60186397 1.        ]]


In [4]:
print(edges)

{0: [12608, 14881, 6049, 3749, 6309, 8426, 6091, 3914, 14030, 4591, 144, 6834, 12050, 1779, 12210, 8029, 4223], 1: [10273, 36, 3653, 4390, 6533, 2024, 14501, 11310, 11919, 1263, 2318, 10642, 13392, 14452, 7707, 9081, 7419, 5564, 6205, 5054], 2: [4739, 13448, 8724, 13717, 3483, 1564, 4004, 805, 6308, 4136, 8753, 10299, 3903, 10561, 2369, 9286, 3400, 4691, 856, 6884, 2407, 4841, 12910, 9327, 625, 11764], 3: [769, 4101, 4102, 1162, 6282, 10381, 3986, 10516, 13720, 2330, 11425, 3874, 3886, 691, 9139, 2869, 8886, 3002, 11962, 11835, 6718, 11460, 14789, 5957, 7493, 6480, 7510, 4441, 986, 3802, 3294, 11891, 12534, 3704, 9087], 4: [3586, 903, 2570, 2187, 908, 11402, 4628, 13718, 9367, 793, 5018, 11161, 2469, 6311, 13355, 10540, 8239, 5552, 3637, 9654, 5559, 4538, 322, 1989, 581, 12999, 14021, 6346, 973, 8400, 1361, 2000, 338, 1625, 602, 4704, 7521, 225, 5736, 3945, 116, 2299], 5: [13059, 6276, 8196, 7046, 1418, 5013, 11542, 1945, 12319, 7846, 10923, 5423, 2736, 5041, 14641, 14003, 437, 441, 99

In [60]:
import numpy as np
import random

num_fields = 21
num_nodes = 5000
predicted_idx = 0
sensitive_idx = 1
predicted_sensitive_rate = 0.75
mid_idx = int(num_nodes/2)

nodes = np.zeros((num_nodes, num_fields))
nodes[mid_idx:,0] = 1
nodes[:int(mid_idx * (1 - predicted_sensitive_rate)),sensitive_idx] = 1
nodes[mid_idx:int(mid_idx + mid_idx * predicted_sensitive_rate),sensitive_idx] = 1

np.random.shuffle(nodes)

for i in range(2,num_fields):
    prob_pos = max(0.0, min(1.0, np.random.normal(0.5, 0.25, None)))
    prob_neg = max(0.0, min(1.0, np.random.normal(0.5, 0.25, None)))
    pos_choices = random.choices([0, 1], weights=(1 - prob_pos, prob_pos), k=int(num_nodes / 2))
    neg_choices = random.choices([0, 1], weights=(1 - prob_neg, prob_neg), k=int(num_nodes / 2))
    neg_counter = 0
    pos_counter = 0
    for j in range(num_nodes):
        if nodes[j][predicted_idx] == 0:
            nodes[j][i] = neg_choices[neg_counter]
            neg_counter += 1
        else:
            nodes[j][i] = pos_choices[pos_counter]
            pos_counter += 1

edges = []
for i in range(num_nodes):
    num_neither = np.random.randint(0, 5)
    num_predicted = np.random.randint(0, 10)
    num_sensitive = np.random.randint(0, 5)
    num_both = np.random.randint(0, 10)
    cur_neither = 0
    cur_predicted = 0
    cur_sensitive = 0
    cur_both = 0
    node_idxs = list(range(num_nodes))
    random.shuffle(node_idxs)
    for j in node_idxs:
        if i != j:
            if nodes[i][predicted_idx] == nodes[j][predicted_idx]:
                if nodes[i][sensitive_idx] == nodes[j][sensitive_idx]:
                    if cur_both < num_both:
                        edges.append(f'{i}\t{j}')
                        cur_both += 1
                else:
                    if cur_predicted < num_predicted:
                        edges.append(f'{i}\t{j}')
                        cur_predicted += 1
            else:
                if nodes[i][sensitive_idx] == nodes[j][sensitive_idx]:
                    if cur_sensitive < num_sensitive:
                        edges.append(f'{i}\t{j}')
                        cur_sensitive += 1
                else:
                    if cur_neither < num_neither:
                        edges.append(f'{i}\t{j}')
                        cur_neither += 1
                    

In [61]:
#with open('../../dataset/matt-synthetic/matt-synthetic_edges.txt', 'w') as write_file:
#    for edge in edges:
#        write_file.write(edge + '\n')
#        
#with open('../../dataset/matt-synthetic/matt-synthetic.csv', 'w') as write_file:
#    first_line = 'PREDICTED,SENSITIVE'
#    for i in range(num_fields - 2):
#        first_line += f',FIELD_{i}'
#    write_file.write(first_line + '\n')
#    for node in nodes:
#        node_str = []
#        for item in node.tolist():
#            node_str.append(str(int(item)))
#        write_file.write(','.join(node_str) + '\n')

In [71]:
edges = []
with open('../../dataset/matt-synthetic/matt-synthetic_edges.txt', 'r') as read_file:
    lines = read_file.readlines()
    for line in lines:
        edges.append(line.strip())

nodes = []
with open('../../dataset/matt-synthetic/matt-synthetic.csv', 'r') as read_file:
    lines = read_file.readlines()
    for line in lines[1:]:
        node_str = line.strip().split(',')
        node = []
        for item in node_str:
            node.append(float(item))
        nodes.append(np.array(node))
nodes = np.array(nodes)

print(nodes.shape[0], len(edges))

5000 64934


In [67]:
#### COMPUTE NODE CORRELATIONS ####
import numpy as np
import scipy.stats as stats

def get_cramer_v(data):
    #Chi-squared test statistic, sample size, and minimum of rows and columns
    x2 = stats.chi2_contingency(data, correction=False)[0]
    n = np.sum(data)
    minDim = min(data.shape)-1

    #calculate Cramer's V 
    v = np.sqrt((x2/n) / minDim)
    return v

def prep_data(fields, protecteds):
    data = np.zeros((2, max(fields) + 1))
    for i, protected in enumerate(protecteds):
        field = fields[i]
        data[protected][field] += 1
            
    return data

for i in range(1, num_fields):  
    data = prep_data(nodes[:,0].astype(int).tolist(), nodes[:,i].astype(int).tolist()) ## both must be list of int
    v = get_cramer_v(data)
    print(v)


0.5
0.03833989743324358
0.2654081828960896
0.05136133899809188
0.06593889554281573
0.06307476320501859
0.6171175121079819
0.029627172367773274
0.4758163542415319
0.3237298079123276
0.07124415085674712
0.37345633068240097
0.10400852072704124
0.3359220585203847
0.5634918425793681
0.46612705060152626
0.18624644376732358
0.017866261801089856
0.2017421831746416
0.6476068549965585


In [72]:
#### COMPUTE EDGE CORRELATIONS ####

sims = []
for i in range(num_fields):
    sims.append([])
with open('../../dataset/matt-synthetic/edges.txt', 'r') as txt_file:
    lines = txt_file.readlines()
    for line in lines:
        src_node = int(line.strip().split('\t')[0])
        dst_node = int(line.strip().split('\t')[1])

        for field in range(num_fields):
            sims[field].append(int(nodes[src_node][field] == nodes[dst_node][field]))
            
for sim in sims:
    print(np.array(sim).mean())

0.49845872816531567
0.5028542071012673
0.5612893404878792
0.5018837766868364
0.5040529740837996
0.5503672413136964
0.9394337253111086
0.6012672679529627
0.5148609049739316
0.5077063591734217
0.5399398713704
0.5059367507706359
0.5132244929025384
0.49902956958556915
0.5775012368230772
0.6385432126955132
0.5191612436731743
0.5440499295962249
0.5128629600030444
0.5169539901815275
0.5704798873539597
